In [5]:
gas_factor = 30

In [6]:
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
import numpy as np
import math

# !!! Determinism !!!
np.random.seed(42)

file_list = [("BLS12", "./bls12/miller_loop_and_final_exp_parallel_5000.csv"),
            ("BN", "./bn/miller_loop_and_final_exp_parallel_5000.csv")]

def get_dfs(files):
    results = []
    for file in files:
        (name, path) = file;
        df = pd.read_csv(path)
        df = df[df["x_is_negative"] == 1.0]
        df["gas"] = df["run_microseconds"].apply(lambda x: math.ceil(x*gas_factor))
        df.drop("run_microseconds", axis = 1, inplace = True);
        df.drop("group_limbs", axis = 1, inplace = True);
        results.append(df)
        
    return results

In [17]:
dataframes = get_dfs(file_list)

In [18]:
dataframes[0].head()

,x_bit_length,x_hamming_weight,modulus_limbs,num_pairs,x_is_negative,gas
0,22,2,4,2,1,60750
1,22,2,7,2,1,138900
2,22,2,4,4,1,59940
3,22,2,8,2,1,168630
4,22,2,10,2,1,237510


In [19]:
dataframes[1].head()

,six_u_plus_two_bit_length,six_u_plus_two_hamming,modulus_limbs,num_pairs,x_is_negative,x_bit_length,x_hamming_weight,gas
0,83,79,4,2,1,81,79,135600
1,83,79,4,4,1,81,79,188220
2,83,79,4,8,1,81,79,294390
3,38,20,7,2,1,36,20,159990
4,83,79,4,16,1,81,79,522600


In [10]:
bls12_one_off = pd.read_csv("./bls12/one_off_results.csv")

bls12_one_off.head(15)

,modulus_limbs,gas
0,4,30180
1,5,43650
2,6,61080
3,7,81270
4,8,110790
5,9,147090
6,10,182490
7,11,223560
8,12,277620
9,13,332160


In [11]:
bn_one_off = pd.read_csv("./bn/one_off_results.csv")

bn_one_off.head(15)

,modulus_limbs,gas
0,4,32100
1,5,47430
2,6,66600
3,7,89220
4,8,122400
5,9,164310
6,10,204090
7,11,252780
8,12,315420
9,13,380400


In [36]:
def correct_for_one_off(df, one_offs):
    new_df = df.copy()
    new_df["gas_corrected"] = new_df.apply(lambda x: x["gas"] - one_offs[one_offs["modulus_limbs"] == x["modulus_limbs"]]["gas"].array[0], axis=1)
    new_df.drop("gas", axis = 1, inplace = True)
    leftovers = new_df[new_df["gas_corrected"] <= 0]
    print("Leftover rows (corrected gas < 0, noise artifacts) = {}".format(leftovers.shape[0]))
    
    new_df = new_df[new_df["gas_corrected"] > 0].copy()
    print("Data rows (corrected gas > 0, reasonable (even if noisy)) = {}".format(new_df.shape[0]))
    
    return new_df

In [37]:
bls12_corrected = correct_for_one_off(dataframes[0], bls12_one_off)
bls12_corrected.head(15)

Leftover rows (corrected gas < 0, noise artifacts) = 964
Data rows (corrected gas < 0, reasonable (even if noisy)) = 259036


,x_bit_length,x_hamming_weight,modulus_limbs,num_pairs,x_is_negative,gas_corrected
0,22,2,4,2,1,30570
1,22,2,7,2,1,57630
2,22,2,4,4,1,29760
3,22,2,8,2,1,57840
4,22,2,10,2,1,55020


In [38]:
bn_corrected = correct_for_one_off(dataframes[1], bn_one_off)
bn_corrected.head(15)

Leftover rows (corrected gas < 0, noise artifacts) = 687
Data rows (corrected gas < 0, reasonable (even if noisy)) = 259311


,six_u_plus_two_bit_length,six_u_plus_two_hamming,modulus_limbs,num_pairs,x_is_negative,x_bit_length,x_hamming_weight,gas_corrected
0,83,79,4,2,1,81,79,103500
1,83,79,4,4,1,81,79,156120
2,83,79,4,8,1,81,79,262290
3,38,20,7,2,1,36,20,70770
4,83,79,4,16,1,81,79,490500


In [39]:
from sklearn import linear_model
from sklearn.linear_model import Lasso
from scipy.optimize import nnls

def factor_out_final_exp_bls(df, skip_bad_fits = False):
    df_final_exps = pd.DataFrame(columns = df.columns);
    df_final_exps.drop("num_pairs", axis = 1, inplace = True);
    df_final_exps.drop("gas_corrected", axis = 1, inplace = True);
    df_final_exps["final_exp_gas"] = 0.0

    df_miller_loops = pd.DataFrame(columns = df.columns);
    df_miller_loops.drop("num_pairs", axis = 1, inplace = True);
    df_miller_loops.drop("gas_corrected", axis = 1, inplace = True);
    df_miller_loops["single_pair_miller_gas"] = 0.0

    min_score = 1.0

    for k, g in df.groupby(['modulus_limbs', "x_bit_length", "x_hamming_weight"]):
        
        model = Lasso(alpha=0.0001,precompute=True,max_iter=1000,
            positive=True, random_state=9999, selection='random')
        model.fit(g["num_pairs"][:, np.newaxis], g["gas_corrected"][:, np.newaxis])
        
        score = model.score(g["num_pairs"][:, np.newaxis], g["gas_corrected"][:, np.newaxis])
        if score < min_score:
            min_score = score
            
        if score < 0.85 and skip_bad_fits:
#             print(g["num_pairs"])
#             print(g["run_microseconds"])
            continue
            
        slope = model.coef_[0];
        intercept = model.intercept_[0];
        
        if slope <= 1 or intercept <= 1:
            continue
            
        g_miller = g.iloc[0].copy()
        g_miller.drop("gas_corrected", inplace = True)
        g_final_exp = g.iloc[0].copy()
        g_final_exp.drop("gas_corrected", inplace = True)

#         g_miller["single_pair_miller_time"] = model.coef_[0][0];
        g_miller["single_pair_miller_gas"] = slope;
        
        g_final_exp["final_exp_gas"] = intercept;

        g_miller.drop("num_pairs", inplace = True)
        g_final_exp.drop("num_pairs", inplace = True)

        df_miller_loops = df_miller_loops.append(g_miller, verify_integrity=True)
        df_final_exps = df_final_exps.append(g_final_exp, verify_integrity=True)
        
    print("Minimal final exp fitting score = {}".format(min_score))
        
    return (df_miller_loops, df_final_exps)

def factor_out_final_exp_bn(df, skip_bad_fits = False):
    df_final_exps = pd.DataFrame(columns = df.columns);
    df_final_exps.drop("num_pairs", axis = 1, inplace = True);
    df_final_exps.drop("gas_corrected", axis = 1, inplace = True);
    df_final_exps["final_exp_gas"] = 0.0

    df_miller_loops = pd.DataFrame(columns = df.columns);
    df_miller_loops.drop("num_pairs", axis = 1, inplace = True);
    df_miller_loops.drop("gas_corrected", axis = 1, inplace = True);
    df_miller_loops["single_pair_miller_gas"] = 0.0

    min_score = 1.0

    for k, g in df.groupby(['modulus_limbs', "x_bit_length", "x_hamming_weight", "six_u_plus_two_bit_length", "six_u_plus_two_hamming"]):
#     for k,g in df.groupby(np.arange(len(df))//group_by):
#         reg = linear_model.LinearRegression(fit_intercept = True)
#         model = reg.fit(g["num_pairs"][:, np.newaxis], g["run_microseconds"][:, np.newaxis])
        
        model = Lasso(alpha=0.0001,precompute=True,max_iter=1000,
            positive=True, random_state=9999, selection='random')
        model.fit(g["num_pairs"][:, np.newaxis], g["gas_corrected"][:, np.newaxis])
        
        score = model.score(g["num_pairs"][:, np.newaxis], g["gas_corrected"][:, np.newaxis])
        if score < min_score:
            min_score = score
            
        if score < 0.85 and skip_bad_fits:
#             print(g["num_pairs"])
#             print(g["run_microseconds"])
            continue
            
        slope = model.coef_[0];
        intercept = model.intercept_[0];
        
        if slope <= 1 or intercept <= 1:
            continue
            
        g_miller = g.iloc[0].copy()
        g_miller.drop("gas_corrected", inplace = True)
        g_final_exp = g.iloc[0].copy()
        g_final_exp.drop("gas_corrected", inplace = True)

#         g_miller["single_pair_miller_time"] = model.coef_[0][0];
        g_miller["single_pair_miller_gas"] = slope;
        
        g_final_exp["final_exp_gas"] = intercept;

        g_miller.drop("num_pairs", inplace = True)
        g_final_exp.drop("num_pairs", inplace = True)

        df_miller_loops = df_miller_loops.append(g_miller, verify_integrity=True)
        df_final_exps = df_final_exps.append(g_final_exp, verify_integrity=True)
        
    print("Minimal final exp fitting score = {}".format(min_score))
        
    return (df_miller_loops, df_final_exps)

In [40]:
from sklearn.model_selection import train_test_split

def split_df(df):
    train, test = train_test_split(
        df, test_size=0.10, random_state=42)
    
    print("Train samples {}, test samples {}".format(len(train), len(test)))
    
    return (train, test)

In [41]:
from sklearn import linear_model
from sklearn.linear_model import Lasso

from sklearn.metrics import max_error, mean_absolute_error, r2_score

def pretty_print_polynomial(poly, model, variable_names):
    terms = []

    for term_idx in range(0, poly.powers_.shape[0]):
        coeff = model.coef_[term_idx]
        if coeff == 0:
            continue
        coeff = np.around(coeff, decimals=6)
        subparts = []
        coeff_string = "{}".format(coeff)
        subparts.append(coeff_string)
        for variable_idx in range(0, poly.powers_.shape[1]):
            power = poly.powers_[term_idx, variable_idx]
            if power != 0:
                if power == 1:
                    term_string = '{}'.format(variable_names[variable_idx])
                    subparts.append(term_string)
                else:
                    term_string = '{}^{}'.format(variable_names[variable_idx], power)
                    subparts.append(term_string)
        if len(subparts) != 0:
            joined = " * ".join(subparts)
            terms.append(joined)

    polynomial_string = " + ".join(terms)
    print(polynomial_string)


In [42]:
def analyze_manual_poly(df, features_description, target, trunc_limit = 0.001, degree = 2):
    
    new_df = df.copy()
    features = []
    for feature in features_description:
        name, max_power = feature
        for i in range(1, max_power+1):
            subname = "{}^{}".format(name, i)
            new_df[subname] = new_df[name].apply(lambda x: x**i)
            features.append(subname)
            
    print(features)
            
    poly = PolynomialFeatures(degree = degree, interaction_only=True, include_bias = False)
        
    train, test = split_df(new_df)

    X_train = train[features]
    Y_train = train[target]
    
    X_train = poly.fit_transform(X_train)

    lin = Lasso(alpha=0.0001,precompute=True, max_iter=100000, fit_intercept=False,
                positive=True, random_state=9999, selection='random')
    lin.fit(X_train, Y_train)
    
    print("Intercept = {}".format(lin.intercept_))

    print("score on training set {}".format(lin.score(X_train, Y_train)))

    X_test = test[features]
    Y_test = test[target]
    
    X_test = poly.fit_transform(X_test)

    print("score on test set {}".format(lin.score(X_test, Y_test)))
    
    y_true = Y_test
    y_pred = lin.predict(X_test)

    print("Model accuracy before manual truncation of coefficients")
    print("Max absolute error {} microseconds".format(max_error(y_true, y_pred)))
    print("Mean absolute error {} microseconds".format(mean_absolute_error(y_true, y_pred)))
    print("R2 score = {}".format(r2_score(y_true, y_pred)))

    coeffs = lin.coef_.copy()
    for k in range(0, coeffs.shape[0]):
        c = coeffs[k]
        if c < trunc_limit:
            coeffs[k] = 0.0

    lin.coef_ = coeffs

    y_true = Y_test
    y_pred = lin.predict(X_test)

    print("Truncating coefficients lower than {}".format(trunc_limit))
    print("Model accuracy after manual truncation of coefficients")
    print("Max absolute error {} microseconds".format(max_error(y_true, y_pred)))
    print("Mean absolute error {} microseconds".format(mean_absolute_error(y_true, y_pred)))
    print("R2 score = {}".format(r2_score(y_true, y_pred)))
    
    pretty_print_polynomial(poly, lin, features)
    
    return lin

In [49]:
def analyze_bls12_alt(df, trunc_limit = 0.001, modulus_power = 12):
    (miller, final_exp) = factor_out_final_exp_bls(df)
    print("Fitting miller loop price")

    model_miller = analyze_manual_poly(miller, [
        ("x_bit_length", 1),
        ("x_hamming_weight", 1),
        ("modulus_limbs", modulus_power)], "single_pair_miller_gas", trunc_limit = trunc_limit, degree = 2)
    
    print("Fitting final exp price")
    model_final_exp = analyze_manual_poly(final_exp, [
        ("x_bit_length", 1),
        ("x_hamming_weight", 1),
        ("modulus_limbs", modulus_power)], "final_exp_gas", trunc_limit = trunc_limit, degree = 2)
    
    return (model_miller, model_final_exp)

In [73]:
model_modulus_max_power = 6

In [74]:
model_multiplication_factor = 1000

In [75]:
truncation_limit = 1.0 / model_multiplication_factor

In [76]:
(bls_miller, bls_final_exp) = analyze_bls12_alt(bls12_corrected, trunc_limit = truncation_limit, modulus_power = model_modulus_max_power)

Minimal final exp fitting score = 0.0
Fitting miller loop price
['x_bit_length^1', 'x_hamming_weight^1', 'modulus_limbs^1', 'modulus_limbs^2', 'modulus_limbs^3', 'modulus_limbs^4', 'modulus_limbs^5', 'modulus_limbs^6']
Train samples 38958, test samples 4329
Intercept = 0.0
score on training set 0.9823433545446876
score on test set 0.9830688333641393
Model accuracy before manual truncation of coefficients
Max absolute error 75846.40783366759 microseconds
Mean absolute error 5143.712650915447 microseconds
R2 score = 0.9830688333641394
Truncating coefficients lower than 0.001
Model accuracy after manual truncation of coefficients
Max absolute error 75846.40783366759 microseconds
Mean absolute error 5143.712650915447 microseconds
R2 score = 0.9830688333641394
29.837819 * x_bit_length^1 * modulus_limbs^1 + 3.994773 * x_bit_length^1 * modulus_limbs^2 + 25.301013 * x_hamming_weight^1 * modulus_limbs^1 + 4.905582 * x_hamming_weight^1 * modulus_limbs^2
Fitting final exp price
['x_bit_length^1',

In [77]:
def analyze_bn_alt(df, trunc_limit = 0.001, modulus_power = 12):
    (miller, final_exp) = factor_out_final_exp_bn(df)
    print("Fitting miller loop price")

    model_miller = analyze_manual_poly(miller, [
        ("six_u_plus_two_bit_length", 1),
        ("six_u_plus_two_hamming", 1),
        ("modulus_limbs", modulus_power)], "single_pair_miller_gas", trunc_limit = trunc_limit, degree = 2)
    
    print("Fitting final exp price")
    model_final_exp = analyze_manual_poly(final_exp, [
        ("x_bit_length", 1),
        ("x_hamming_weight", 1),
        ("modulus_limbs", modulus_power)], "final_exp_gas", trunc_limit = trunc_limit, degree = 2)
    
    return (model_miller, model_final_exp)

In [78]:
(bn_miller, bn_final_exp) = analyze_bn_alt(bn_corrected, trunc_limit = truncation_limit, modulus_power = model_modulus_max_power)

Minimal final exp fitting score = 0.0008305217552782018
Fitting miller loop price
['six_u_plus_two_bit_length^1', 'six_u_plus_two_hamming^1', 'modulus_limbs^1', 'modulus_limbs^2', 'modulus_limbs^3', 'modulus_limbs^4', 'modulus_limbs^5', 'modulus_limbs^6']
Train samples 38773, test samples 4309
Intercept = 0.0
score on training set 0.9700731357990067
score on test set 0.9706262986490389
Model accuracy before manual truncation of coefficients
Max absolute error 226050.42663546858 microseconds
Mean absolute error 6073.24359836941 microseconds
R2 score = 0.9706262986490389
Truncating coefficients lower than 0.001
Model accuracy after manual truncation of coefficients
Max absolute error 226050.42663546858 microseconds
Mean absolute error 6073.24359836941 microseconds
R2 score = 0.9706262986490389
6.993634 * modulus_limbs^2 + 31.765174 * six_u_plus_two_bit_length^1 * modulus_limbs^1 + 4.141652 * six_u_plus_two_bit_length^1 * modulus_limbs^2 + 22.398625 * six_u_plus_two_hamming^1 * modulus_li

In [79]:
import json

def process_polynomial_model(model, features_description):
#     expect only 2nd order cross terms
    features = []
    features_encoded_as_int = []
    for (feature_index, feature) in enumerate(features_description):
        name, max_power = feature
        for power in range(1, max_power+1):
            subname = "{}^{}".format(name, power)
            features.append(subname)
            features_encoded_as_int.append((feature_index, power))
            
    poly = PolynomialFeatures(degree = 2, interaction_only=True, include_bias = False)
    
    _ = poly.fit_transform([[0.0]*len(features)])
            
    unrolled_coeffs = []
    for term_idx in range(0, poly.powers_.shape[0]):
        coeff = model.coef_[term_idx]
        coeff = math.ceil(coeff * model_multiplication_factor)
        if coeff == 0:
            continue
        subparts = []
        for variable_idx in range(0, poly.powers_.shape[1]):
            power = poly.powers_[term_idx, variable_idx]
            if power != 0:
                if power == 1:
                    subparts.append(features_encoded_as_int[variable_idx])
#                     term_string = '{}'.format(variable_names[variable_idx])
#                     subparts.append(term_string)
                else:
#                     we do not expect terms like x*x due to features structure
                    assert(False)
#                     term_string = '{}^{}'.format(variable_names[variable_idx], power)
#                     subparts.append(term_string)
        if len(subparts) != 0:
            unrolled_coeffs.append((coeff, subparts))
            
    compressed_terms = []
    for term in unrolled_coeffs:
        coeff, terms = term
        if len(terms) < 2:
            compressed_terms.append((coeff, terms))
        else:
#         there are only two terms max
            if terms[0][0] == terms[1][0]:
                compressed_terms.append((coeff, [(terms[0][0], terms[0][1] + terms[1][1])]))
            else:
                compressed_terms.append((coeff, terms))
                
    deduped_terms = []
    for i in range(0, len(compressed_terms)):
        coeff, terms = compressed_terms[i]
        if len(terms) != 1:
            deduped_terms.append((coeff, terms))
            continue
            
        deduped = False
        for j in range(0, len(deduped_terms)):
            existing_coeff, existing_terms = deduped_terms[j]
            if len(existing_terms) != 1:
                continue
            if terms[0][0] == existing_terms[0][0] and terms[0][1] == existing_terms[0][1]:
                print("Dedup")
                existing_coeff += coeff
                deduped_terms[j] = (existing_coeff, existing_terms)
                deduped = True
                
        if deduped == False:
            deduped_terms.append((coeff, terms))
    
    return deduped_terms

def serialize_bn_bls_model(one_offs, miller_model, miller_features, final_exp_model, final_exp_features, filename):
    result = {}
    subres = []
    for (index, row) in one_offs.iterrows():
        subres.append([int(math.floor(row["modulus_limbs"])), math.ceil(row["gas"]*model_multiplication_factor)])
    result["one_off"] = subres
    result["multiplier"] = model_multiplication_factor
    result["miller_features"] = miller_features
    result["miller"] = process_polynomial_model(miller_model, miller_features)
    result["final_exp_features"] = final_exp_features
    result["final_exp"] = process_polynomial_model(final_exp_model, final_exp_features)
    
    with open(filename, 'w') as outfile:
        json.dump(result, outfile)

In [80]:
serialize_bn_bls_model(bls12_one_off, bls_miller,[
        ("x_bit_length", 1),
        ("x_hamming_weight", 1),
        ("modulus_limbs", model_modulus_max_power)],
                       bls_final_exp,[
        ("x_bit_length", 1),
        ("x_hamming_weight", 1),
        ("modulus_limbs", model_modulus_max_power)], "bls12_model.json")

In [81]:
serialize_bn_bls_model(bn_one_off, bn_miller,[
        ("six_u_plus_two_bit_length", 1),
        ("six_u_plus_two_hamming", 1),
        ("modulus_limbs", model_modulus_max_power)],
                       bn_final_exp,[
        ("x_bit_length", 1),
        ("x_hamming_weight", 1),
        ("modulus_limbs", model_modulus_max_power)], "bn_model.json")